In [ ]:
!pip install py7zr
!pip install pymorphy2
!pip install nltk
!pip install razdel
!pip install transformers

In [3]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForPreTraining
import numpy as np
import pandas as pd
import pymorphy2
from razdel import sentenize
import nltk
import py7zr
from razdel import tokenize
from nltk import sent_tokenize
from tqdm import tqdm
from nltk.corpus import stopwords
import re

In [5]:
with py7zr.SevenZipFile('/content/drive/MyDrive/dataset/news2017.7z', mode='r') as z:
    z.extract(targets='news2017/1.csv')

In [4]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")

In [5]:
df = pd.read_csv("/content/news2017/1.csv")

In [6]:
df.dropna(inplace=True)

In [7]:
stopwords = stopwords.words('russian')

In [14]:
vocabulary = {}
cache = {}
banned_tags = {"NPRO", "PRED", "PREP","CONJ", "PRCL", "INTJ", "PNCT", "UNKN", "NUMB"}
analyzer = pymorphy2.MorphAnalyzer()
for idx in tqdm(range(df.shape[0])):
    if (idx == 10):
        break
    string_to_parse = df.iloc[idx].values[1]
    string_to_parse = re.sub(' *\n\n *', '. ', string_to_parse)
    new_str = re.sub(' +|\n *\n| *\xa0| *\n', ' ', string_to_parse)
    sents = sentenize(new_str)
    for sent in sents:
        d = tokenizer(sent.text, return_offsets_mapping=True, return_tensors='pt')
        offset_mapping = d.pop('offset_mapping').detach().numpy()[0]
        sent_vect = model(**d)[0].detach().numpy()[0]
        words = tokenize(sent.text.lower())
        # старт и стоп отвечают за позиции токенов рассматриваемого слова
        start = 1
        for word in words:
            w = word.text
            while start < len(offset_mapping) - 1 and offset_mapping[start][0] != word.start:
                start += 1
            stop = start
            while stop < len(offset_mapping) - 1 and offset_mapping[stop][1] != word.stop:
                stop += 1
            if w not in cache:
                pm_info = analyzer.parse(w)[0]
                cache[w] = pm_info
                w_syn = pm_info
            else:
                w_syn = cache[w]
            if str(w_syn.tag).split(',')[0] not in banned_tags:
                normal_form = w_syn.normal_form
                if normal_form not in stopwords:
                    if normal_form not in vocabulary:
                        vocabulary[normal_form] = []
                    vocabulary[normal_form].append(sent_vect[start:stop+1].mean(axis=0))
            start = stop


  0%|          | 10/655988 [01:03<1160:49:44,  6.37s/it]


In [8]:
tokenizer_s = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model_s = AutoModelForPreTraining.from_pretrained("cointegrated/rubert-tiny")